In [1]:
import tensorflow as tf
import os
import tiktoken
import re

2023-06-03 20:37:42.870595: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-03 20:37:42.904299: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
block_size = 128
batch_size = 64

In [3]:
# Byte Pair Encoder
enc = tiktoken.get_encoding("p50k_base")

In [4]:
data = []
for file in os.listdir("./data"):
    print(f"Working on file : {file}")
    with open(f"./data/{file}", "r") as f:
        data.append(f.read())

Working on file : republic


In [5]:
# concatenate
data = "\n".join(data)

In [6]:
data = re.sub("\n+", "\n", data)

In [7]:
data[:500]

'I went down yesterday to the Piraeus with Glaucon the son of Ariston, that I might offer up my prayers to the goddess (Bendis, the Thracian Artemis.); and also because I wanted to see in what manner they would celebrate the festival, which was a new thing. I was delighted with the procession of the inhabitants; but that of the Thracians was equally, if not more, beautiful. When we had finished our prayers and viewed the spectacle, we turned in the direction of the city; and at that instant Polem'

In [8]:
data = tf.convert_to_tensor(enc.encode(data), tf.float16)

2023-06-03 20:37:53.713517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-03 20:37:53.732175: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-03 20:37:53.732314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-03 20:37:53.732823: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the

In [9]:
data.shape

TensorShape([147085])

In [10]:
x = tf.stack([data[idx:idx+block_size] for idx in range(len(data)-block_size)])
y = tf.stack([data[idx+1:idx+block_size+1] for idx in range(len(data)-block_size)])

In [11]:
x.shape

TensorShape([146957, 128])

In [12]:
data = tf.data.Dataset.from_tensor_slices((x, y))  

In [13]:
len(data)

146957

In [14]:
def train_val_test_split(dataset, train_split, val_split, test_split):
    dataset_size = len(dataset)
    dataset = dataset.shuffle(dataset_size)
    train_size = int(train_split * dataset_size)
    val_size = int(val_split * dataset_size)
    
    train_data = dataset.take(train_size)
    val_data = dataset.skip(train_size).take(val_size)
    test_data = dataset.skip(train_size+val_size)
    return train_data, val_data, test_data

In [15]:
train, val, test = train_val_test_split(data, 0.7, 0.2, 0.1)

In [16]:
# For the train:
# We split it into 4 shards
offset = len(train) // 4
offset

25717

In [17]:
shard_list = []
for i in range(4):
    if i == 3:
        shard_list.append(train.skip(offset * i).shuffle(offset).batch(batch_size))
    else:
        shard_list.append(train.skip(offset * i).take(offset).shuffle(offset).batch(batch_size))

In [18]:
len(shard_list[0]) == len(shard_list[1]) == len(shard_list[2])

True

In [19]:
len(shard_list[3])

402

In [20]:
shard_list

[<BatchDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.float16, name=None), TensorSpec(shape=(None, 128), dtype=tf.float16, name=None))>,
 <BatchDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.float16, name=None), TensorSpec(shape=(None, 128), dtype=tf.float16, name=None))>,
 <BatchDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.float16, name=None), TensorSpec(shape=(None, 128), dtype=tf.float16, name=None))>,
 <BatchDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.float16, name=None), TensorSpec(shape=(None, 128), dtype=tf.float16, name=None))>]

In [21]:
for ind, ds in enumerate(shard_list):
    ds.save(f"./shards/shard_{ind}")

In [22]:
val = val.batch(batch_size)
val.save("./shards/val_shard")

In [23]:
test = test.batch(batch_size)
test.save("./shards/test_shard")

In [24]:
path = "./shards"
shards = []
for file in os.listdir(path):
    if file.startswith("shard"):
        shards.append(tf.data.Dataset.load(f"{path}/{file}"))

In [25]:
shards

[<_LoadDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.float16, name=None), TensorSpec(shape=(None, 128), dtype=tf.float16, name=None))>,
 <_LoadDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.float16, name=None), TensorSpec(shape=(None, 128), dtype=tf.float16, name=None))>,
 <_LoadDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.float16, name=None), TensorSpec(shape=(None, 128), dtype=tf.float16, name=None))>,
 <_LoadDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.float16, name=None), TensorSpec(shape=(None, 128), dtype=tf.float16, name=None))>]

In [26]:
next(iter(shards[0]))

(<tf.Tensor: shape=(64, 128), dtype=float16, numpy=
 array([[1.980e+02, 2.436e+03, 5.230e+02, ..., 6.120e+02, 3.180e+02,
         2.570e+02],
        [1.980e+02, 3.600e+04, 4.070e+02, ..., 3.180e+02, 4.368e+03,
         1.100e+01],
        [7.740e+03, 8.200e+01, 3.290e+02, ..., 2.900e+02, 3.510e+02,
         5.840e+02],
        ...,
        [3.390e+02, 5.310e+02, 1.300e+01, ..., 2.900e+02, 3.290e+02,
         2.620e+02],
        [3.790e+02, 1.752e+03, 1.100e+01, ..., 2.860e+02, 5.970e+02,
         3.296e+03],
        [3.560e+02, 4.070e+02, 6.350e+02, ..., 5.340e+02, 1.103e+03,
         4.460e+03]], dtype=float16)>,
 <tf.Tensor: shape=(64, 128), dtype=float16, numpy=
 array([[2.4360e+03, 5.2300e+02, 3.0000e+01, ..., 3.1800e+02, 2.5700e+02,
         1.3650e+03],
        [3.6000e+04, 4.0700e+02, 1.3000e+01, ..., 4.3680e+03, 1.1000e+01,
         3.1800e+03],
        [8.2000e+01, 3.2900e+02, 2.6200e+02, ..., 3.5100e+02, 5.8400e+02,
         1.8290e+03],
        ...,
        [5.3100e+02, 1.3

In [27]:
len(shards[0])

402